In [1]:
%pylab qt5

Populating the interactive namespace from numpy and matplotlib


In [2]:
import scipy

In [3]:
from importlib import reload
import lib2
from IPython.display import clear_output
from lib import data_management as dm
from time import sleep
import drivers
from lib2.IQPulseSequence import *
from drivers.KeysightAWG import *

warnings.filterwarnings("ignore",".*GUI is implemented.*")
mpl.rcParams['grid.color'] = 'k'
mpl.rcParams['grid.linestyle'] = ':'
mpl.rcParams['grid.linewidth'] = 0.5

In [4]:
if_freq = 50e6

calibrations = (dm.load_IQMX_calibration_database("Xmons_Nb_bandage_res_mixer", -10))
ro_cal = calibrations.get(frozenset(dict(lo_power=7, ssb_power=-22, lo_frequency=7.535e9, 
                                      if_frequency=0, waveform_resolution=1).items()))
ro_pb = PulseBuilder(ro_cal)

calibrations = (dm.load_IQMX_calibration_database("Xmons_Nb_bandage_qub_mixer", -6))
q_cal = calibrations.get(frozenset(dict(lo_power=7, ssb_power=-22, lo_frequency=6.16e9+if_freq, 
                                      if_frequency=if_freq, waveform_resolution=0.1).items()))
q_pb = PulseBuilder(q_cal)

In [6]:
q_cal.get_optimization_results()

({'dc_offsets': array([ 0.00710647, -0.03581821]),
  'dc_offsets_open': None,
  'if_amplitudes': array([ 0.24658357,  0.2248116 ]),
  'if_offsets': array([ 0.00663281, -0.03863291]),
  'if_phase': array([-1.60709686])},
 {'dc': -80.516433715820313,
  'if': [-22.21625328063965, -59.436492919921875, -81.97730255126953]})

In [7]:
ro_awg = KeysightAWG("AWG3")

In [8]:
q_awg = KeysightAWG("AWG2")

In [358]:
from drivers.Keysight_DSOX2014 import *

In [359]:
dso = Keysight_DSOX2014("DSO2014")

______________

### Resonators scan areas

In [30]:
scan_areas = [(7529945247.2623634, 7579945247.2623634),
  (7541330816.5408268, 7591330816.5408268),
  (7552773888.6944351, 7602773888.6944351),
  (7564561978.0989046, 7614561978.0989046),
  (7574164958.2479124, 7624164958.2479124)]

In [31]:
scan_areas

[(7529945247.262363, 7579945247.262363),
 (7541330816.540827, 7591330816.540827),
 (7552773888.694435, 7602773888.694435),
 (7564561978.098905, 7614561978.098905),
 (7574164958.247912, 7624164958.247912)]

# Spectroscopy

## Single-tone spectroscopy

In [9]:
import lib2.TwoToneSpectroscopy

In [10]:
#reload(lib2.SingleToneSpectroscopy)
from lib2.SingleToneSpectroscopy import *

In [11]:
ro_awg.output_continuous_wave(10e6, 1, offset=0)

In [361]:
q_pb.add_sine_pulse(100).add_zero_pulse(100).add_dc_pulse(100,1).build().plot()


In [368]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100,5).build())
q_awg.output_pulse_sequence(q_pb.add_zero_pulse(100).build())

In [369]:
STS = SingleToneSpectroscopy("III-anticrossing", "Xmons_Nb_bandage", vna = 'vna1', src = 'yok3')
vna_parameters = {"bandwidth":500, "freq_limits":(7.52e9, 7.55e9), "nop":1001, "power":7, "averages":1, "CW":False}
STS.set_fixed_parameters(vna_parameters)
currents = linspace(1.3e-3, 1.42e-3, 31)
STS._src.set_current(currents[0])
sleep(1)
STS.set_swept_parameters({'current': (STS._src.set_current, currents)})
result = STS.launch()
STS._src.set_current(0);
result.visualize();

vna1 is already initialized
yok3 is already initialized
Started at:  2017-08-08 17:01:05.835428
Time left: 0 h 0 m 42.53 s, [current: 1.34e-03]: , average cycle time: 2.03 s       

In [16]:
result.visualize();

In [13]:
result.save()

## Two-tone spectroscopy

In [13]:
# reload(lib2.Measurement)
#reload(lib2.TwoToneSpectroscopyBase)
#reload(lib2.TwoToneSpectroscopy)
from lib2.TwoToneSpectroscopy import *

In [371]:
q_pb.add_sine_pulse(100, amplitude=5).build().plot()

In [12]:
q_pb.add_zero_pulse(100).add_dc_pulse(800, 5).add_zero_pulse(100).build().plot()
ro_pb.add_zero_pulse(100).add_dc_pulse(800,5).add_zero_pulse(100).build().plot()

In [39]:
ro_awg.output_continuous_wave(50e6, 2)

In [40]:
ro_awg.output_pulse_sequence(ro_pb.add_zero_pulse(10).add_dc_pulse(980,5).add_zero_pulse(10).build())
q_awg.output_pulse_sequence(q_pb.add_zero_pulse(10).add_dc_pulse(980, 5).add_zero_pulse(10).build())

In [30]:
ro_awg._visainstrument.write(":ARM:SOUR1 IMM")

(16, <StatusCode.success: 0>)

#### Qubit spectrum

In [14]:
TTS = FluxTwoToneSpectroscopy("I-two-tone", "Xmons_Nb_bandage", vna_name="vna1", 
                              mw_src_name="mxg", current_src_name="yok3")
vna_parameters = {"bandwidth":200, "freq_limits":(7.52e9, 7.55e9), "nop":50, "sweep_type":"CW", "power":7, "averages":1}
mw_src_parameters = {"power":7}
mw_src_frequencies = linspace(6.12e9, 6.18e9, 201)
currents = linspace(1.34e-3, 1.36e-3, 11)
TTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, currents)

The device vna1 is detected as PNA-L1
The device mxg is detected as MXG
The device yok3 is detected as GS210_3
Detecting a resonator within provided frequency range of the VNA (7520000000.0, 7550000000.0)                    at qubit's sweet spot (1.35 mA)
Detected frequency is 7.53511 GHz, at 3.01 mU and -64.94 degrees


In [57]:
result = TTS.launch()

Started at:  2017-08-08 19:44:11.881157
Time left: 0 h 9 m 34.17 s, parameter value: 1.340e-03, average cycle time: 0.28 s           

In [ ]:
result.get_data()['data'].shape

In [ ]:
TTS._current_src

In [ ]:
result.set_plot_range(1.7,1.9)

In [40]:
result.visualize();

In [55]:
result.save()

##### TTS result processing and fitting

In [72]:
angles = np.angle(result.get_data()['data'])
curs = result.get_data()['Current [A]']
freqs = result.get_data()['frequency']

In [76]:
TTS_pts = []
for i,ang_slice in enumerate(angles[:]):
    TTS_pts.append(freqs[np.argmax(ang_slice)])

In [237]:
def f_spectr(x, x0, A, delt):
    return (-1*np.sqrt((A*(x-x0))**2 + (delt)**2)) + 2*delt

In [243]:
plt.plot(curs, f_spectr(np.array(curs), 1.35e-3, 1e14, 6e9));

In [248]:
popt, pcov = scipy.optimize.curve_fit(f_spectr, curs[1:-2], np.array(TTS_pts[1:-2]), p0=[1.35e-3, 1e14, 6.15e9]);
plt.plot(curs[1:-2], np.array(TTS_pts[1:-2]), 'o', curs[1:-2], f_spectr(curs[1:-2], *popt));
plt.grid(True)

___________________

In [ ]:
result._name = "5.5GHz-two-tone-w-loop_-0.5mA-and-coil"

In [ ]:
TTS._mw_src.set_output_state("OFF")

#### Linewidth from excitation power

In [15]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100, 5).build())

In [58]:
PTS = PowerTwoToneSpectroscopy("VI-powerscan", "Xmons_Nb_bandage", vna_name='vna1', mw_src_name='mxg', current_src_name='yok3')
vna_parameters = {"bandwidth":200, "freq_limits":(7.52e9, 7.55e9), "nop":50, "power":7, "sweep_type":"CW","averages":1}
mw_src_parameters = {}
mw_src_frequencies = linspace(6.14e9, 6.18e9,401)
mw_src_powers = linspace(7, 7.1, 2)
current = 1.36e-3
PTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, mw_src_powers, current)
PTS._measurement_result.set_phase_units("deg")

vna1 is already initialized
mxg is already initialized
yok3 is already initialized
Detecting a resonator within provided frequency range of the VNA (7520000000.0, 7550000000.0)                    at current of 1.36 mA
Detected frequency is 7.53511 GHz, at 4.53 mU and -123.33 degrees


In [59]:
result = PTS.launch()

Started at:  2017-08-08 19:45:43.804415
Time left: 0 h 3 m 26.8 s, parameter value: 7.000e+00, average cycle time: 0.28 s           

In [336]:
result.set_plot_range(min_abs=0.19, max_abs=0.21, min_phas=65, max_phas=85)

In [320]:
result._plot_limits_fixed = True

In [51]:
result.visualize();

In [50]:
TTS._actual_devices.keys()

dict_keys(['vna1', 'mxg', 'yok3', 'awg3', 'awg2'])

In [ ]:
TTS._construct_fixed_parameters()

In [ ]:
vna.set_parameters(PTS._pre_measurement_vna_parameters)

In [338]:
result.save()

In [ ]:
mxg.set_frequency(7.898e9)
mxg.set_power(-30)
mxg.set_output_state("OFF")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
ground_state_sdata = vna.get_sdata()

mxg.set_output_state("ON")
vna.prepare_for_stb(); vna.sweep_single(); vna.wait_for_stb()
mixed_state_sdata = vna.get_sdata()

In [ ]:
plot(abs(ground_state_sdata))
plot(abs(mixed_state_sdata))
plot((abs(ground_state_sdata)[10:]+abs(ground_state_sdata)[:-10])/2)

#### AC Stark

In [ ]:
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(100).build())
q_awg.output_pulse_sequence(q_pb.add_dc_pulse(100, 5).build())

In [ ]:
ASTS = AcStarkTwoToneSpectroscopy("VI-ac-stark", "Xmon Al BMSTU S444 2")
vna_parameters = {"bandwidth":1000, "freq_limits":(7040e6, 7052.5e6), "nop":1, "power":-35, "averages":100}
mw_src_parameters = {"power":-15, "frequency":-1}
mw_src_frequencies = linspace(7.40e9, 7.46e9, 100)
vna_powers = linspace(-30, 0, 31)
current = 2.355e-3
ASTS.setup_control_parameters(vna_parameters, mw_src_parameters, mw_src_frequencies, vna_powers, current)

In [ ]:
result = ASTS.launch()

In [87]:
result.visualize()

(<matplotlib.figure.Figure at 0x11962518>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x0000000011B92A20>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000001235A898>], dtype=object),
 (<matplotlib.axes._axes.Axes at 0x123bcd30>,
  <matplotlib.axes._axes.Axes at 0x12422160>))

In [86]:
result.save()

In [ ]:
TTS.close_devs("mxg")

__________________________

# Time-domain measurements

### Check mixer calibration

In [16]:
q_lo = TTS._mw_src
q_lo.set_output_state("ON")
q_lo.set_frequency(6.214e9)
q_lo.set_power(7)

In [55]:
q_lo.visa_instr.close()

In [17]:
TTS._vna.set_nop(1)
TTS._vna.set_power(7)
TTS._vna.set_center(7.5351e9)
TTS._vna.sweep_continuous()

In [18]:
q_awg.output_pulse_sequence(q_pb.add_zero_pulse(20).build())
ro_awg.output_pulse_sequence(ro_pb.add_zero_pulse(20).build())

In [19]:
q_awg.output_pulse_sequence(q_pb.add_sine_pulse(200,0).build())
ro_awg.output_pulse_sequence(ro_pb.add_dc_pulse(200,5).build())

In [28]:
q_cal.get_radiation_parameters()

{'if_frequency': 50000000.0,
 'lo_frequency': 6210000000.0,
 'lo_power': 7,
 'ssb_power': -22,
 'waveform_resolution': 0.1}

### Set qubit frequency

In [ ]:
TTS._current_src.set_current(-2.6e-4)

In [23]:
q_freq

6164000000.0

In [22]:
q_freq = 6.164e9

In [ ]:
q_freq = q_freq+0.03e6
q_freq

### Rabi 

In [25]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveRabiOscillations)
from lib2.DispersiveRabiOscillations import DispersiveRabiOscillations

In [29]:
DRO = DispersiveRabiOscillations("I-rabi", "Xmons_Nb_bandage",
            vna_name="vna1", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7.525e9, 7.545e9), "nop":10, "sweep_type":"LIN", "averages":2}
exc_frequency = q_freq
excitation_durations = linspace(0, 700, 71)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":8000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRO.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
DRO.set_swept_parameters(excitation_durations)

vna1 is already initialized
awg3 is already initialized
awg2 is already initialized
mxg is already initialized
Detecting a resonator within provided frequency range of the VNA (7525000000.0, 7545000000.0)                    
Detected frequency is 7.53515 GHz, at 0.44 mU and -91.56 degrees


In [30]:
result = DRO.launch()

Started at:  2017-08-08 20:51:40.548728
Time left: 0 h 3 m 20.69 s, [excitation_duration: 3.00e+01]: , average cycle time: 3.0 s        

In [ ]:
result.visualize();

In [ ]:
result.save()

In [ ]:
pi_pulse_duration = result.get_pi_pulse_duration()*1e3
pi_pulse_duration

In [ ]:
pi_pulse_duration = 67.949209045107793

### Decay

In [ ]:
# reload(lib2.VNATimeResolvedMeasurement)
# reload(lib2.DispersiveDecay)
from lib2.DispersiveDecay import *

In [ ]:
DD = DispersiveDecay("VI-decay", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg', )
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
q_frequency = q_freq
readout_delays = linspace(0, 5000,20)
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":10000, "pi_pulse_duration":pi_pulse_duration}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DD.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, q_frequency, rabi_sequence_parameters)
DD.set_swept_parameters(readout_delays)

In [ ]:
dd_result = DD.launch()

In [ ]:
dd_result.visualize();

In [ ]:
dd_result.save()

### Ramsey

In [ ]:
# reload(lib2.IQPulseSequence)
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveRamsey)
from lib2.DispersiveRamsey import *

In [ ]:
DR = DispersiveRamsey("VI-ramsey", "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                                                                  q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
ramsey_delays = linspace(0, 2000, 100)
exc_frequency = q_freq-5e6
rabi_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, rabi_sequence_parameters)
DR.set_swept_parameters(ramsey_delays)

In [ ]:
dr_result = DR.launch()

In [ ]:
dr_result.visualize();

In [ ]:
dr_result.save()

### Echo

In [ ]:
# reload(lib2.VNATimeResolvedMeasurement)
# from lib2.VNATimeResolvedMeasurement import *
# reload(lib2.DispersiveHahnEcho)
from lib2.DispersiveHahnEcho import *

In [ ]:
DHN = DispersiveHahnEcho("VI-hahn-echo", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":20, "averages":2}
q_frequency = q_freq
echo_delays = linspace(10, 7000, 20)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DHN.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, q_frequency, sequence_parameters)
DHN.set_swept_parameters(echo_delays)

In [ ]:
dhn_result = DHN.launch()

In [ ]:
dhn_result.visualize();

In [ ]:
dhn_result.save()

In [ ]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-hahn-echo").visualize()

### Ramsey fringes

In [ ]:
# reload(lib2.Measurement)
# reload(drivers.KeysightAWG)
reload(lib2.VNATimeResolvedDispersiveMeasurement)
reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
reload(lib2.DispersiveRamseyFringes)
from lib2.DispersiveRamseyFringes import *

In [ ]:
DRF = DispersiveRamseyFringes("VI-ramsey-fringes", "Xmon Al BMSTU S444 2",
                                      vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "averages":2}
q_frequency = 7.89939e9
excitation_freqs = linspace(q_frequency-10e6, q_frequency+10e6, 200)
ramsey_delays = linspace(10, 5000, 200)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":10000, "half_pi_pulse_duration":pi_pulse_duration/2}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRF.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, sequence_parameters)
DRF.set_swept_parameters(ramsey_delays, excitation_freqs)

In [ ]:
drf_result = DRF.launch()

In [ ]:
drf_result.visualize()

In [ ]:
drf_result.save()

### Rabi shevrons

In [ ]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement2D)
# reload(lib2.DispersiveRabiChevrons)
from lib2.DispersiveRabiChevrons import *

In [ ]:
DRC = DispersiveRabiChevrons("VI-rabi-shevrons", "Xmon Al BMSTU S444 2",
                                      vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":1, "averages":2}
q_frequency = 7.89939e9
excitation_freqs = linspace(q_frequency-10e6, q_frequency+10e6, 200)
ramsey_delays = linspace(10, 5000, 200)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DRC.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, sequence_parameters)
DRC.set_swept_parameters(ramsey_delays, excitation_freqs)

In [ ]:
drc_result = DRC.launch()

In [ ]:
drc_result.visualize();

In [ ]:
drc_result.save()

### Dispersive shift measurement

In [ ]:
reload(lib2.VNATimeResolvedDispersiveMeasurement)
reload(lib2.VNATimeResolvedDispersiveMeasurement2D)
reload(lib2.TimeResolvedDispersiveShiftSpectroscopy)
from lib2.TimeResolvedDispersiveShiftSpectroscopy import *

In [ ]:
DSS = TimeResolvedDispersiveShiftSpectroscopy("VI-rabi-dispersive-shift-exact-q-freq", "Xmon Al BMSTU S444 2",
                                      vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":100, "averages":2}
exc_frequency = 7.89919e9
excitation_durations = linspace(1, 3000, 100)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, "repetition_period":10000}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DSS.set_fixed_parameters(vna_parameters, q_frequency, ro_awg_params, q_awg_params, sequence_parameters)
DSS.set_swept_parameters(excitation_durations)

In [ ]:
dss_result = DSS.launch()

In [ ]:
dss_result.save()

In [ ]:
MeasurementResult.load("Xmon Al BMSTU S444 2", "VI-rabi-dispersive-shift").visualize();

In [ ]:
dss_result.visualize();

______________________________________________

# Pulse optimization

## Amplified phase error

In [ ]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersiveAPE)
from lib2.DispersiveAPE import *

### Rectangular pulses

In [ ]:
for i in range(0, 11, 1):
    print(i)
    DAPE = DispersiveAPE("VI-APE-%d-pseudo-I-pulses"%i, "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                                q_awg_name="awg2", q_lo_name='mxg')
    vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10**(1.7*i/10), "averages":2}
    ramsey_angles = linspace(-4*pi, 4*pi, 100)
    exc_frequency = q_freq
    ro_awg_params =  {"calibration":ro_cal}
    q_awg_params = {"calibration":q_cal}
    APE_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000}
    APE_sequence_parameters.update({"half_pi_pulse_duration":pi_pulse_duration/2, 
                                    "pseudo_I_pulses_count":i, "modulating_window":'rectangular',
                                    "excitation_amplitude":1, "padding":5})
    DAPE.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, APE_sequence_parameters)
    DAPE.set_swept_parameters(ramsey_angles)
    dape_result = DAPE.launch()
    dape_result.save()

In [ ]:
dape_result = DAPE.launch()

In [ ]:
dape_result.visualize();

In [ ]:
dape_result.save()

### Gaussian pulses

#### Rabi

In [ ]:
# reload(lib2.VNATimeResolvedDispersiveMeasurement)
# reload(lib2.VNATimeResolvedDispersiveMeasurement1D)
# reload(lib2.DispersivePiPulseAmplitudeCalibration)
from lib2.DispersivePiPulseAmplitudeCalibration import *

In [ ]:
DPPAC = DispersivePiPulseAmplitudeCalibration("VI-pi-amp-cal", "Xmon Al BMSTU S444 2",
            vna_name="vna2", ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
exc_frequency = q_freq
excitation_amps = linspace(0, 3, 20)
sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000,
                            "repetition_period":8000, "excitation_duration":50, "modulating_window":"gaussian"}
ro_awg_params =  {"calibration":ro_cal}
q_awg_params = {"calibration":q_cal}
DPPAC.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, sequence_parameters)
DPPAC.set_swept_parameters(excitation_amps)

In [ ]:
dppac_result = DPPAC.launch()

In [ ]:
dppac_result.visualize();

In [ ]:
pi_pulse_amplitude = 2.28
pi_pulse_amplitude

#### Ramsey 

In [ ]:
from lib2.DispersiveRamsey import *

In [ ]:
DR = DispersiveRamsey("VI-ramsey", "Xmon Al BMSTU S444 2", vna_name="vna2", 
                                          ro_awg_name="awg3", q_awg_name="awg2", q_lo_name='mxg')
vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10, "averages":2}
ramsey_delays = linspace(0, 2000, 100)
exc_frequency = q_freq-5e6
ramsey_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                              "excitation_amplitude":pi_pulse_amplitude/2, "modulating_window":"gaussian",
                              "repetition_period":8000, "half_pi_pulse_duration":50}
DR.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, ramsey_sequence_parameters)
DR.set_swept_parameters(ramsey_delays)

In [ ]:
drg_result = DR.launch()

In [ ]:
drg_result.visualize();

#### APE

In [ ]:
for i in range(0, 11, 1):
    print(i)
    DAPE = DispersiveAPE("VI-APE-gaussian-%d-pseudo-I-pulses"%i, "Xmon Al BMSTU S444 2", vna_name="vna2", ro_awg_name="awg3", 
                                                q_awg_name="awg2", q_lo_name='mxg')
    vna_parameters = {"bandwidth":10, "freq_limits":(7040e6, 7052.5e6), "nop":10**(1.7*i/10), "averages":3}
    ramsey_angles = linspace(-4*pi, 4*pi, 100)
    exc_frequency = q_freq
    ro_awg_params =  {"calibration":ro_cal}
    q_awg_params = {"calibration":q_cal}
    APE_sequence_parameters = {"awg_trigger_reaction_delay":400, "readout_duration":1000, 
                            "repetition_period":8000, "padding":5,
                            "half_pi_pulse_duration":50, "modulating_window":"gaussian",
                            "excitation_amplitude":pi_pulse_amplitude/2, "pseudo_I_pulses_count":i}
    DAPE.set_fixed_parameters(vna_parameters, ro_awg_params, q_awg_params, exc_frequency, APE_sequence_parameters)
    DAPE.set_swept_parameters(ramsey_angles)
    dape_result = DAPE.launch()
    dape_result.save()
    clear_output()

In [ ]:
dape_result.visualize()

---------------------------

In [ ]:
reload(lib2.IQPulseSequence)
from lib2.IQPulseSequence import *

In [ ]:
pb = PulseBuilder(q_cal)

In [ ]:
APE_sequence_parameters["ramsey_angle"]=0
APE_sequence_parameters["pseudo_I_pulses_count"] = 1
APE_sequence_parameters["excitation_amplitude"] = 1

In [ ]:
APE_sequence_parameters

In [ ]:
PulseBuilder.build_dispersive_APE_sequences(pb, ro_pb, APE_sequence_parameters)[0].plot()
pb.add_zero_pulse(400).add_sine_pulse(65).add_sine_pulse(50).add_zero_pulse(15).build().plot()
plt.xlim(300,600)

In [ ]:
# pb.add_zero_pulse(10).add_sine_pulse(30).add_zero_pulse(10).build().plot()
pb.add_zero_pulse(10).add_sine_pulse(50, window=("gaussian", {"amplitude":10})).add_zero_pulse(10).build().plot()
plt.grid()

In [ ]:
rect = pb.add_zero_pulse(100).add_sine_pulse(300, amplitude=1).add_zero_pulse(1000).build()

In [ ]:
gaus = pb.add_zero_pulse(100).add_sine_pulse(300, amplitude=1, window="gaussian").add_zero_pulse(1000).build()

In [ ]:
rect.plot()
gaus.plot()

In [ ]:
q_awg.output_pulse_sequence(gaus)

In [ ]:
exp(-1/8)

In [ ]:
X = linspace(-1,1)

In [ ]:
plot(X, exp(-(3*X)**2/2**2))

In [ ]:
from resonator_tools import circuit

In [ ]:
circuit.notch_port()